In [4]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Run necessary background code and generate baseline

In [5]:
# Background functions and calculations of adjustment factors
execfile('import_packages.py')
execfile('read_in_data.py')
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')
execfile('usercode_taxcalc.py')

All packages successfully imported
Data imports complete
Beginning CCR model
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated


In [6]:
execfile('gen_baseline.py')

Taxable income calculated
Investment matrices constructed
Corporate depreciation calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
Noncorporate tax rates calculated
Baseline complete


# Specify tax reform

In [7]:
btax_dict1 = {
    2017: {
        'tau_c': 0.357,
        #'tau_nc': 0.0,
        #'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
iit_params_ref = {}
execfile('reform_implementation.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
All parameters updated


# Calculate the investment response

In [8]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr_c': 0.0,
    'inv_eatr_nc': 0.0,
    'mne_share_c': 0.0,
    'mne_share_nc': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': -0.2
}
execfile('responses.py')

In [9]:
execfile('mini_combined.py')
response_results = inv_response(2017)

In [10]:
response_results

,Asset,deltaIc2014,deltaInc2014,MPKc2014,MPKnc2014,deltaIc2015,deltaInc2015,MPKc2015,MPKnc2015,deltaIc2016,...,MPKc2025,MPKnc2025,deltaIc2026,deltaInc2026,MPKc2026,MPKnc2026,deltaIc2027,deltaInc2027,MPKc2027,MPKnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300354,0.298111,0.0,0.0,0.300354,0.297631,0.0,0.0,0.300354,0.299022
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.139446,0.126643,0.0,0.0,0.139446,0.126618,0.0,0.0,0.139446,0.126604
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179538,0.173637,0.0,0.0,0.179538,0.173737,0.0,0.0,0.179538,0.173830
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121538,0.116298,0.0,0.0,0.121538,0.116359,0.0,0.0,0.121538,0.116407
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.448545,0.436229,0.0,0.0,0.448545,0.436186,0.0,0.0,0.448545,0.436182
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.237326,0.224333,0.0,0.0,0.237326,0.224283,0.0,0.0,0.237326,0.224252
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124725,0.116436,0.0,0.0,0.124725,0.116442,0.0,0.0,0.124725,0.116447
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.214061,0.205874,0.0,0.0,0.214061,0.205926,0.0,0.0,0.214061,0.205976


In [11]:
legal_response(2017)
print rescale_corp
print rescale_noncorp

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
[ 1.          1.          1.          0.99688554  0.99688725  0.99689807
  0.99690444  0.99691677  0.99692747  0.99693563  0.9969435   0.99695052
  0.9969568   0.99696178]
[ 1.          1.          1.          1.00430348  1.00430977  1.00433368
  1.00434836  1.00437461  1.0043975   1.00441308  1.00442827  1.00444316
  1.00445573  1.00447038]


# Run other responses and recalculate corporate tax revenue

In [12]:
execfile('gen_reform.py')

New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete


### Check capital results

In [13]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,917.231276,7679.962002,946.192485,10299.957146,634.436143,864.209963,2017
4,955.593579,8116.470387,985.766063,10816.562257,670.495787,818.416277,2018
5,990.217329,8569.171682,1021.483044,11353.902282,706.990464,796.786114,2019
6,1024.371760,9031.852889,1056.715888,11904.934610,743.285142,633.703941,2020
7,1062.969553,9502.713125,1096.532391,12467.344037,779.383070,714.240564,2021
8,1104.991364,9983.169525,1139.881023,13041.677609,815.830353,783.130690,2022
9,1148.877976,10477.935528,1185.153338,13633.248921,853.234937,841.394459,2023


In [14]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [15]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,304.764438,3010.964246,319.495743,7438.239333,193.893312,337.496067,2017
4,317.512357,3182.113826,332.859854,7709.889404,204.914618,324.228449,2018
5,329.020955,3360.481555,344.924740,7994.295774,216.102022,318.504585,2019
6,340.372299,3543.841851,356.824771,8287.857209,227.265370,262.046683,2020
7,353.202211,3731.519299,370.274839,8589.060009,238.404922,291.987940,2021
8,367.169593,3923.739823,384.917358,8896.475175,249.672563,317.934449,2022
9,381.755158,4122.154130,400.207941,9212.561184,261.245863,340.182823,2023


In [16]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


### Check combined results

In [17]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2445.993351,884.575577,231.441998,34.306208,1295.669568,0.021643,0.347,421.555664,101.582376,4.457941,3.700228,320.731000
1,2015,2520.645171,905.241147,243.981131,34.855388,1336.567506,0.021643,0.347,434.862110,91.794692,4.529305,3.799722,343.797000
2,2016,2327.856171,830.188301,248.001094,34.816996,1214.849781,0.021643,0.347,395.260349,96.318143,4.524316,3.895522,299.571000
3,2017,2420.802112,864.209963,267.390681,34.843347,1254.358121,0.021643,0.357,420.658261,99.624700,4.400913,3.965698,321.468775
4,2018,2436.216510,818.416277,294.636829,34.909968,1288.253435,0.021643,0.357,432.025305,96.299358,4.409327,4.022852,336.112422
5,2019,2512.417712,796.786114,327.897563,35.239831,1352.494204,0.021643,0.357,453.568921,97.682144,4.450991,4.073644,356.264123
6,2020,2511.997040,633.703941,361.313620,35.745934,1481.233544,0.021643,0.357,496.742609,103.233192,4.514915,4.123219,393.901113
7,2021,2648.130415,714.240564,392.473246,36.486898,1504.929708,0.021643,0.357,504.689293,108.667862,4.608502,4.174703,396.455230
8,2022,2785.282852,783.130690,421.211400,37.489084,1543.451678,0.021643,0.357,517.607920,114.453781,4.735084,4.231710,403.657513
9,2023,2908.651106,841.394459,447.101917,38.853973,1581.300757,0.021643,0.357,530.300888,121.246587,4.907477,4.297842,409.663936


In [18]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.021643,0.347,1295.669568,34.306208,884.575577,231.441998,2445.993351
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.021643,0.347,1336.567506,34.855388,905.241147,243.981131,2520.645171
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.021643,0.347,1214.849781,34.816996,830.188301,248.001094,2327.856171
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.021643,0.347,1258.241448,34.952204,866.909916,268.261581,2428.365149
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.021643,0.347,1292.254685,35.018974,820.971756,295.578104,2443.823519
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.021643,0.347,1356.690115,35.349483,799.265383,328.930336,2520.235317
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.021643,0.347,1485.825688,35.856932,635.671702,362.442904,2519.797226
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.021643,0.347,1509.579935,36.599743,716.449543,393.691242,2656.320464
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.021643,0.347,1548.206251,37.604625,785.544298,422.511919,2793.867094
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.021643,0.347,1586.160018,38.973402,843.980733,448.477548,2917.591701


In [19]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print "Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3))
print "Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3))
print "Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3))

Total corporate tax revenue change, 2014-2027: 154.75
Total corporate tax revenue change, 2017-2026: 138.147
Total corporate tax revenue change, 2018-2027: 143.287


In [20]:
for i in compare:
    print i

5.68434188608e-14
-1.13686837722e-13
-1.13686837722e-13
11.4627749289
11.769421765
12.3651231852
13.5511125251
13.7932295998
14.1675134684
14.5359364341
14.993012702
15.4895670285
16.0192806876
16.6025370891


## Calculate percent changes in investment income and pass-through income

In [21]:
execfile('multipliers_for_taxcalc.py')

In [22]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1105.741653,2014,1416.393849,1416.393849,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1092.837295,2015,1386.540753,1386.540753,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,915.439816,2016,1179.897668,1179.897668,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,949.148567,2017,1213.679328,1197.506513,0.986675,0.996754,1.004300,1.004316,1.004295,1.004307
4,988.844086,2018,1151.313026,1134.971472,0.985806,0.996815,1.004308,1.004317,1.004305,1.004312
5,1024.661477,2019,1138.215655,1121.265562,0.985108,0.996860,1.004333,1.004338,1.004331,1.004335
6,1059.997174,2020,1031.411151,1013.497165,0.982632,0.996884,1.004348,1.004351,1.004347,1.004349
7,1099.923709,2021,1098.173702,1079.818870,0.983286,0.996906,1.004374,1.004376,1.004374,1.004375
8,1143.394137,2022,1163.520433,1144.583585,0.983725,0.996922,1.004397,1.004399,1.004397,1.004398
9,1188.796255,2023,1218.128548,1198.650316,0.984010,0.996933,1.004413,1.004414,1.004413,1.004413


## Pass changes to Tax-Calculator

In [23]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [24]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

2.35991903467e-14
7.53214734706e-14
0.0
-2.93347824103
-3.38337081323
-3.63605645216
-4.63532710243
-4.53921310234
-4.52382416494
-4.56049307302
-4.63727499828
-4.72546182464
-4.8675357347
-5.08472239085
-47.5267578976


In [25]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,2.359919e-14,2014,5.684342e-14,8.044261e-14
1,7.532147e-14,2015,-1.136868e-13,-3.836536e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-2.933478e+00,2017,1.146277e+01,8.529297e+00
4,-3.383371e+00,2018,1.176942e+01,8.386051e+00
5,-3.636056e+00,2019,1.236512e+01,8.729067e+00
6,-4.635327e+00,2020,1.355111e+01,8.915785e+00
7,-4.539213e+00,2021,1.379323e+01,9.254016e+00
8,-4.523824e+00,2022,1.416751e+01,9.643689e+00
9,-4.560493e+00,2023,1.453594e+01,9.975443e+00


In [26]:
print "Total tax revenue change, 2014-2027: " + str(np.round(sum(ModelResults['RevenueChange']), 3))
print "Total tax revenue change, 2017-2026: " + str(np.round(sum(ModelResults['RevenueChange'][3:13]), 3))
print "Total tax revenue change, 2018-2027: " + str(np.round(sum(ModelResults['RevenueChange'][4:]), 3))

Total tax revenue change, 2014-2027: 107.223
Total tax revenue change, 2017-2026: 95.705
Total tax revenue change, 2018-2027: 98.693


In [27]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

8.52929668784
8.38605095179
8.72906673305
8.91578542264
9.25401649744
9.64368930347
9.97544336107
10.3557377037
10.7641052039
11.1517449529
11.5178146983


95.7049368178
